In [203]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# Changes the number of columns diplayed (default is 20)
pd.options.display.max_columns = 250
# Changes the number of rows diplayed (default is 60)
pd.options.display.max_rows = 250
pd.set_option("display.precision", 3)


   
SolarData = pd.read_csv("DATA/SolarData.txt", sep=",")
SolarData = SolarData[SolarData['On/Off'] == 1]
WeatherData = pd.read_csv("DATA/WeatherData_Combined.txt", sep=",", header=0)
WindData = pd.read_csv("DATA/WindData.txt", sep=",")
WindData = WindData[WindData['On/Off'] == 1]
LandPrice = pd.read_csv("DATA/Prices of land.csv", sep=',')
#solar
merged_df = pd.merge(SolarData, WeatherData,on=['Date','location'], how='inner')
df = merged_df
df = df[(df['type'] == 2)]
mean_value_solar = df['kWh produced'].mean()
print("Solar Mean value per unit:", mean_value_solar)
#wind
merged_df = pd.merge(WindData, WeatherData,on=['Date','location'], how='inner')
df = merged_df
df = df[(df['type'] == "C")]
mean_value_wind = df['kWh produced'].mean() * 1000
print("Wind Mean value per unit:", mean_value_wind)








price_wind = 1950000
price_solar = 70000

space_wind = 50000
space_solar = 500
print()
Power_space_solar = mean_value_solar / space_solar
print("solar power m2:", Power_space_solar)

Power_space_wind = mean_value_wind / space_wind
print("wind power m2:", Power_space_wind)
print()

Power_price_solar = mean_value_solar / price_solar
print("solar power E:", Power_price_solar)

Power_price_wind = mean_value_wind / price_wind
print("wind power E:", Power_price_wind)
print()


Power_price_space_solar = mean_value_solar / price_solar/ space_solar
print("solar power per E * m2:", Power_price_space_solar)

Power_price_space_wind = mean_value_wind / price_wind/ space_wind
print("wind power per E * m2 ", Power_price_space_wind)
print()


# so data is swaped places and Wind mill B might be better ????

# 0.01419


Solar Mean value per unit: 236.95827585756783
Wind Mean value per unit: 27683.84417630112

solar power m2: 0.47391655171513564
wind power m2: 0.5536768835260224

solar power E: 0.0033851182265366834
wind power E: 0.014196843167333908

solar power per E * m2: 6.770236453073367e-06
wind power per E * m2  2.8393686334667813e-07



In [204]:
#print(LandPrice)
#here we are neglecting the constrains

N_wind_possible = LandPrice['Remaining Capacity ha'] // 5
N_solar_rem = (LandPrice['Remaining Capacity ha'] % 5) / 0.05
LandPrice.insert(4,'Windmills possible' ,N_wind_possible)
LandPrice.insert(5,'Solar remaining' ,N_solar_rem)


In [205]:
#rule correction
#Zwart
LandPrice.at[4, 'Windmills possible'] = 5
LandPrice.at[4, 'Solar remaining'] = 20/0.05 - 391 # -391 makes it fit the budget and that is the best way to reduce the budget while remaining efficiency


#Roodeschool
LandPrice.at[6, 'Windmills possible'] = 0
LandPrice.at[6, 'Solar remaining'] = 85/0.05
#visschershoek
LandPrice.at[10, 'Solar remaining'] = 0
#Kruishoofd
LandPrice.at[11, 'Windmills possible'] = 0
LandPrice.at[11, 'Solar remaining'] = 25/0.05

print(LandPrice)


            Location Additional requirements  Remaining Capacity ha  \
0              Arcen                     NaN                    3.5   
1              Eelde                     NaN                    8.0   
2    Airport Twenthe     max 30 solar panels                   25.0   
3           Lelystad                     NaN                    4.0   
4         Zwartemeer         Max 5 windmills                   45.0   
5         Rhederbrug                     NaN                   70.0   
6        Roodeschool            No windmills                   85.0   
7          Sexbierum         Max 3 windmills                   12.0   
8     Burgervlotbrug   Min 5 windmill type C                   32.0   
9   Hoek van Holland                     NaN                   15.0   
10     Visschershoek         No solar panels                   45.0   
11        Kruishoofd            No windmills                   25.0   

    Price to aqquire  Windmills possible  Solar remaining  
0               

In [206]:

#print(LandPrice)
Price2fillout = (LandPrice['Price to aqquire'] + LandPrice['Windmills possible'] * price_wind + LandPrice['Solar remaining'] * price_solar ) / 1000000
#print(Price2fillout)
energyfillout = LandPrice['Windmills possible'] * \
    mean_value_wind + LandPrice['Solar remaining'] * mean_value_solar

PriceEnergyEfficiency = Price2fillout / energyfillout
print(PriceEnergyEfficiency)

0     2.954e-04
1     1.468e-04
2     7.044e-05
3     2.954e-04
4     9.684e-05
5     8.259e-05
6     3.096e-04
7     1.140e-04
8     9.758e-05
9     8.350e-05
10    8.624e-05
11    3.139e-04
dtype: float64


In [207]:
LandPrice.insert(6,'Fillout price' ,Price2fillout)
LandPrice.insert(7,'Energy fillout' ,energyfillout)
LandPrice.insert(8,'Efficiency' , PriceEnergyEfficiency)
#print(LandPrice)


In [208]:
list = LandPrice.sort_values('Efficiency', ascending = True)
print(list)



            Location Additional requirements  Remaining Capacity ha  \
2    Airport Twenthe     max 30 solar panels                   25.0   
5         Rhederbrug                     NaN                   70.0   
9   Hoek van Holland                     NaN                   15.0   
10     Visschershoek         No solar panels                   45.0   
4         Zwartemeer         Max 5 windmills                   45.0   
8     Burgervlotbrug   Min 5 windmill type C                   32.0   
7          Sexbierum         Max 3 windmills                   12.0   
1              Eelde                     NaN                    8.0   
0              Arcen                     NaN                    3.5   
3           Lelystad                     NaN                    4.0   
6        Roodeschool            No windmills                   85.0   
11        Kruishoofd            No windmills                   25.0   

    Price to aqquire  Windmills possible  Solar remaining  Fillout price  \


In [222]:
Decision1 = list.drop(['Additional requirements', 'Price to aqquire', 'Price to aqquire', 'Remaining Capacity ha', 'Efficiency','Energy fillout',],axis=1)
Decision1 = Decision1.drop([6, 11], axis=0)
print("DECISION 1 : Space efficiency - First filling out land to produce as much energy, than buying next land that has the biggest potential to generate energy the cheapest")

print(Decision1.to_string(index=False))
#Energy_sum = list["Energy fillout"].sum()
print()
dep = list["Energy fillout"].sum()/1000
print("Daily energy production in MWh: %.2f "%dep)
cost = Decision1["Fillout price"].sum()
print("Decision cost: %.2f" %cost, "M€")

DECISION 1 : Space efficiency - First filling out land to produce as much energy, than buying next land that has the biggest potential to generate energy the cheapest
        Location  Windmills possible  Solar remaining  Fillout price
 Airport Twenthe                 5.0              0.0          9.750
      Rhederbrug                14.0              0.0         32.011
Hoek van Holland                 3.0              0.0          6.934
   Visschershoek                 9.0              0.0         21.488
      Zwartemeer                 5.0              9.0         13.611
  Burgervlotbrug                 6.0             40.0         17.134
       Sexbierum                 2.0             40.0          7.392
           Eelde                 1.0             60.0          6.150
           Arcen                 0.0             70.0          4.900
        Lelystad                 0.0             80.0          5.600

Daily energy production in MWh: 1837.93 
Decision cost: 124.97 M€
